In [1]:
import pandas as pd
import requests
import random
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from Credentials import *
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID,
                                                           client_secret= Client_Secret))

# Playlist scarping

In [ ]:
# identifying the playlist we want to use
playlist = sp.user_playlist_tracks("Billboard", "6UeSakyzhiEt4NB3UAd6NQ")

In [ ]:
playlist.keys() # Let's look at items and total:

In [ ]:
#checking how many songs there are
print("Total number of songs in the playlist: ",playlist["total"])

In [2]:
##ESENTIAL
#Extracting the tracks (everything in them) of the playlist 

def get_playlist_tracks(username, playlist_id):
    
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

len(get_playlist_tracks("Billboard", "6UeSakyzhiEt4NB3UAd6NQ"))

100

In [ ]:
#obtaining all of the artists from the playlist

def get_artists_from_playlist(playlist_id):
    tracks_from_playlist = get_playlist_tracks("Billboard", playlist_id)
    
    return list(set(artist for subset in [get_artists_from_track(track["track"]) for track in tracks_from_playlist] for artist in subset))

In [ ]:
# Obtaining the Artist's ID's

    # first we obtain the artists from the tracks
def get_artists_from_track(track):
    return[artist["name"] for artist in track["artists"]]

#--------------------------------------------------------------------
    
    #second we obtain the ids from the artists
def get_artists_ids_from_track(track):
    return[artist["id"] for artist in track["artists"]]

In [ ]:
def get_artists_ids_from_playlist(playlist_id):
    tracks_from_playlist = get_playlist_tracks("Billboard", playlist_id)
    return list(set(artist for subset in [get_artists_ids_from_track(track["track"]) for track in tracks_from_playlist] for artist in subset))

In [ ]:
artists = get_artists_from_playlist("6UeSakyzhiEt4NB3UAd6NQ") # Apply the first function
artists_ids = get_artists_ids_from_playlist("6UeSakyzhiEt4NB3UAd6NQ")
len(artists)
len(artists_ids) # We might have more ids due to artists having the same name

In [ ]:
# Iterate over the playlist's tracks to get all the songs ID's, names, and artists.
    #(Output: Dictionary with key: song ID and values: list(song name, artist).)
    
songs = {
  track["track"]["id"]: [track["track"]["name"], 
  track["track"]["artists"][0]["name"]] 
  for track in playlist["items"]
}

In [ ]:
#we convert "songs into a DataFrame(index = track ID)
songsDF = pd.DataFrame.from_dict(songs, orient = 'index')

In [ ]:
#renaming the columns
songsDF.rename(columns={0: 'Title', 1: 'Artist'}, inplace = True)

In [ ]:
songsDF['Title'] = songsDF['Title'].apply(lambda x: x.split('(')[0].lower())
songsDF.head()

In [3]:
##ESENTIAL

#function to get the title and artist of a song 

def basic_features(username, playlist_id):
    ##identifying the playlist we want to use
    playlist = sp.user_playlist_tracks(username, playlist_id)
    
    ## obtaining the tracks
    get_playlist_tracks(username, playlist_id)
    
    ## Iterate over the playlist's tracks to get all the songs ID's, names, and artists.
    songs = {
      track["track"]["id"]: [track["track"]["name"], 
      track["track"]["artists"][0]["name"]] 
      for track in playlist["items"]}
    
    ##we convert "songs into a DataFrame(index = track ID)
    songsDF = pd.DataFrame.from_dict(songs, orient = 'index')
    #we rename columns
    songsDF.rename(columns={0: 'Title', 1: 'Artist'}, inplace = True)
    
    ## get the title in the right format
    songsDF['Title'] = songsDF['Title'].apply(lambda x: x.split(' (')[0].lower())
    
    return songsDF.head()


In [4]:
##ESENTIAL
basic_features("Billboard", "6UeSakyzhiEt4NB3UAd6NQ")

,Title,Artist
3tjFYV6RSFtuktYl3ZtYcq,mood,24kGoldn
7igeByaBM0MgGsgXtNxDJ7,positions,Ariana Grande
5cYFwgA5dHoqR3xGnlr9ew,i hope,Gabby Barrett
2SAqBLGA283SUiwJ3xOUVI,laugh now cry later,Drake
0VjIjW4GlUZAMYd2vXMi3b,blinding lights,The Weeknd


In [ ]:
# geting the audio features of the songs in a playlist and cvonverting them into a DataFrame

def get_audio_features_df(playlist_id):
    
    track_ids = songsDF.index
    aud_feat = [sp.audio_features(track)[0] for track in track_ids]
    
    return pd.DataFrame(aud_feat)

In [ ]:
featuresDF = get_audio_features_df("6UeSakyzhiEt4NB3UAd6NQ")

featuresDF.head()

In [ ]:
#making id the index of featuresDF

featuresDF.set_index('id', inplace = True)
featuresDF.head()

In [ ]:
#dropping columns

featuresDF.drop(['uri', 'track_href', 'analysis_url', 'time_signature', 'type'], axis = 1, inplace = True)
featuresDF.head()

In [ ]:
# concatenate both DataFrames (get_audio_features_df & SongsDF)

songs = pd.merge(songsDF, featuresDF, right_index=True, left_index=True)
songs.head()

pd.concat([songsDF, featuresDF],axis=1)

In [ ]:
##ESENTIAL

#SUM ALL OF THIS INTO ONE FUNCTION
def hot_features(playlist_id):

## obtaining the audio features of the songs in a playlist and converting them into a DataFrame
    track_ids = songsDF.index
    aud_feat = [sp.audio_features(track)[0] for track in track_ids]
    return pd.DataFrame(aud_feat)
    
##making id the index of featuresDF
    featuresDF.set_index('id', inplace = True)
    
##dropping "usless" columns
    featuresDF.drop(['uri', 'track_href', 'analysis_url', 'time_signature', 'type'], axis = 1, inplace = True)
    
##concatenate both DataFrames... 
    songs = pd.merge(songsDF, featuresDF, right_index=True, left_index=True)
    songs.head()

    pd.concat([songsDF, featuresDF],axis=1)

In [ ]:
##ESENTIAL
hot_features("6UeSakyzhiEt4NB3UAd6NQ")

In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [ ]:
#scaling the features
featuresDF.describe()

In [ ]:
features_prep = StandardScaler().fit_transform(featuresDF)

In [ ]:
pd.DataFrame(features_prep, columns = featuresDF.columns).head()

In [ ]:
# Finding out the optimal number of clusters for the featuers

K = range(2, 20)
inertia = []

for k in K:
    kmeans = KMeans(n_clusters=k,
                    random_state=1234)
    kmeans.fit(features_prep)
    inertia.append(kmeans.inertia_)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,8))
plt.plot(K, inertia, 'bx-')
plt.xlabel('k')
plt.ylabel('inertia')
plt.xticks(np.arange(min(K), max(K)+1, 1.0))
plt.title('Elbow Method showing the optimal k')

In [ ]:
##Creating the Clusters for the features

kmeans = KMeans(n_clusters=9, random_state=42)
kmeans.fit(features_prep)

In [ ]:
# Now that we have our "model", let's get the clusters and how many observations has each cluster.
clusters = kmeans.predict(features_prep)
pd.Series(clusters).value_counts().sort_index()

In [ ]:
#Now let's see to which group was asinged every song.

X_df = pd.DataFrame(featuresDF)
X_df["cluster"] = clusters
X_df.head()

In [ ]:
# summing it all up in a function --> pending

def assign_clusters(features_prep, n_clusters):
    
    kmeans = KMeans(n_clusters, random_state = 42)
    kmeans.fit(features_prep)
    
    clusters = kmeans.predict(features_prep)
    values = pd.Series(clusters).value_counts().sort_index
    
    X_df = pd.DataFrame(featuresDF)
    X_df["cluster"] = clusters
    X_df.head()
    
    return X_df, kmeans

assign_clusters(features_prep, n_clusters)

 ### User Input

In [ ]:
def song_recomendation():
    
    recomendation = input('Enter your song here: ')

    if songs.Title.isin([recomendation]).any() == True:
        sample = random.choice(songs.Title)
        print('Hot song! Here is another HOT recomendation: ')
        print('')
        print(sample)
    
    else:
    ##get the song if, its features and convert them into a DataFrame
        sample2 = sp.search(q= recomendation, type='track')
        track_id = sample2["tracks"]["items"][0]["id"]
        track_feat = sp.audio_features(track_id)
        
        song_features = pd.DataFrame(track_feat)

    ## make dataframe the same in order to predict same cluster
        song_features.drop(['uri', 'track_href', 'analysis_url', 'time_signature', 'type','id'], axis = 1, inplace = True)
        
    ##transform/scale features
        
        song_features_transformed = StandardScaler().fit_transform(song_features)
    
    ##predict cluster
        pcluster = kmeans.predict(song_features_transformed)
        
        #print(pcluster)
    
    ##apply the filter (cluster)
        songsDF_filtered = X_df[X_df['cluster'] == pcluster[0]].sample()
        #print(songsDF_filtered.index)
    
    ##input randomizer

        x = random.choice(X_df.index)
        print("Your song is not very HOT right now... maybe you'll like this one:")
        #songsDF.loc[x]
        print("Title: {}, Artist: {}".format(songsDF.loc[songsDF_filtered.index][['Title','Artist']].values[0][0],
                                         songsDF.loc[songsDF_filtered.index][['Title','Artist']].values[0][1]))
        #print(x)
        #songs.Title.loc[x]
        
        
        

In [ ]:
song_recomendation()